In [1]:
from jampy.quadva import quadva
import astropy.units as u
import numpy as np
import time

In [2]:
def alphax(tau1, 
            y, x, M, sigma, q):
    tau = tau1[:, None]
   
    x_til = x/sigma
    y_til = y/sigma

   
    
    eta = np.sqrt(1 - q**2)
    eta_sq = 1 - q**2
    
    aux = (M/sigma)*(x_til/(np.sqrt(1 - eta_sq*(tau)**2)))
    exp_arg = (tau**2/2)*(x_til**2 + y_til**2/(1-eta_sq*(tau)**2))
    exp = np.exp(-exp_arg)
    

    
    arr = aux*exp    
    
    return tau1*arr.sum(1)



def alphay(tau1,
            y, x, M, sigma, q):
    

    tau = tau1[:, None]    
    x_til = x/sigma
    y_til = y/sigma
    eta = np.sqrt(1 - q**2)
    eta_sq = 1 - q**2
    
    aux = (M/sigma)*(y_til/(np.power(1 - eta_sq*(tau)**2, 3/2)))
    exp_arg = (tau**2/2)*(x_til**2 + y_til**2/(1-eta_sq*(tau)**2))
    exp = np.exp(-exp_arg)
    
    
    arr = aux*exp
    
    return tau1*arr.sum(1)

In [3]:
def MGE_Grid_parameters(grid, quiet=True):
    y0 = np.array([0])
    x0 = np.array([0])
    initial = np.array([y0, x0, M, sigma, q])
    Grid_parameters = np.array([y0, x0, M, sigma, q])

        #Agora realizamos um loop para criar todas as posições:
    for i in range(len(grid)-1): #-1 pois já começamos com uma posição (initial)
        Grid_parameters = np.vstack([Grid_parameters, initial])

        #Agora atualizamos as posições (y,x) e convertemos suas unidades para rad
    Grid_parameters[:, 0] = (grid[:, 0]*u.arcsec).to(u.rad).value
    Grid_parameters[:, 1] = (grid[:, 1]*u.arcsec).to(u.rad).value


        #Class parameter
    Grid_parameters
        
    if quiet is False:
        print("Pyautolens MGE Class successfully initialized!!")
        

    return Grid_parameters

In [4]:
M, sigma, q = np.loadtxt("Input.txt", unpack=True)
grid = np.loadtxt("grid.txt")

In [5]:
#grid = np.random.rand(6000,2)
Grid_parameter = MGE_Grid_parameters(grid)

In [7]:
result_x = np.zeros([len(grid), 3])              #Onde ficarão salvos os resultados da deflexão em x
result_y = np.zeros([len(grid), 3])              #Onde ficarão salvos os resultados da deflexão em y

         
start = time.time()
for i in range(len(grid)):                      #Começo do loop
    result_x[i] = quadva(alphax, [0., 1.], args=(Grid_parameter[i]),epsrel=1e-10)   #Integral em x
    result_y[i] = quadva(alphay, [0., 1.], args=(Grid_parameter[i]),epsrel=1e-10)   #Integral em y
print(time.time()-start)

46.08441758155823


In [7]:
start = time.time()
quadva(alphax, [0., 1.], args=(Grid_parameter[0]),epsrel=1e-10) 
print(time.time() - start)

0.0024788379669189453


## scipy.integrate.quad_vec

In [9]:
from scipy.integrate import quad_vec

y = Grid_parameter[:, 0]
x = Grid_parameter[:, 1]


eta = np.sqrt(1 - q**2)
eta_sq = 1 - q**2

In [105]:
y = Grid_parameter[:, 0]
x = Grid_parameter[:, 1]


eta = np.sqrt(1 - q**2)
eta_sq = 1 - q**2


resultado_x = np.zeros([len(grid), 2])              #Onde ficarão salvos os resultados da deflexão em x
resultado_y = np.zeros([len(grid), 2])              #Onde ficarão salvos os resultados da deflexão em y

start = time.time()
for i in range(10000):
    x_til = x[i]/sigma
    y_til = y[i]/sigma


    ax = lambda tau: tau * np.sum(  (M/sigma)*(x_til/(np.sqrt(1 - eta_sq*(tau)**2)))       *  np.exp( - (tau**2/2)*(x_til**2 + y_til**2/(1-eta_sq*(tau)**2)) )     )

    ay = lambda tau: tau * np.sum(  (M/sigma)*(y_til/(np.power(1 - eta_sq*(tau)**2, 3/2))) *  np.exp( - (tau**2/2)*(x_til**2 + y_til**2/(1-eta_sq*(tau)**2)) )     )

    resultado_x[i] = quad_vec(ax, 0, 1)
    resultado_y[i] = quad_vec(ay, 0, 1)

    
print(time.time() - start)

130.3701205253601


In [17]:
start = time.time()
x_til = x[1]/sigma
y_til = y[1]/sigma


ax = lambda tau: tau * np.sum(  (M/sigma)*(x_til/(np.sqrt(1 - eta_sq*(tau)**2)))       *  np.exp( - (tau**2/2)*(x_til**2 + y_til**2/(1-eta_sq*(tau)**2)) )     )

quad_vec(ax, 0, 1,epsrel=1e-10 )

print(time.time() - start)

0.00878453254699707


In [15]:
start = time.time()
x_til = x[1]/sigma
y_til = y[1]/sigma


ax = lambda tau: tau * np.sum(  (M/sigma)*(x_til/(np.sqrt(1 - eta_sq*(tau)**2)))       *  np.exp( - (tau**2/2)*(x_til**2 + y_til**2/(1-eta_sq*(tau)**2)) )     )

quad_vec(ax, 0, 1, )

print(time.time() - start)

0.008595466613769531


In [121]:
####################################################################################

In [41]:
result_x = np.zeros([len(grid), 2])              #Onde ficarão salvos os resultados da deflexão em x
result_y = np.zeros([len(grid), 2])              #Onde ficarão salvos os resultados da deflexão em y


start = time.time()
for i in range(20):
    result_x = quadrature(alphax, 0., 1., args=(Grid_parameter[i][0], Grid_parameter[i][1], Grid_parameter[i][2],Grid_parameter[i][3],Grid_parameter[i][4]),rtol=1e-10, maxiter=200 )
    result_y = quadrature(alphay, 0., 1., args=(Grid_parameter[i][0], Grid_parameter[i][1], Grid_parameter[i][2],Grid_parameter[i][3],Grid_parameter[i][4]),rtol=1e-10, maxiter=200 )
    
print(time.time() - start)

0.6016504764556885


In [24]:
import numpy as np
import pyquad
import quadpy
Grid_parameter.shape

(17692, 5)

In [119]:
valx = np.zeros(len(Grid_parameter))
valy = np.zeros(len(Grid_parameter))
errx = np.zeros(len(Grid_parameter))
erry = np.zeros(len(Grid_parameter))
start = time.time()
for i in range(10000):

    valx[i], errx[i] = quadpy.quad(alphax, 0.0, 1, args=(Grid_parameter[i]),epsrel=1e-10, limit=100)
    valy[i], erry[i] = quadpy.quad(alphay, 0.0, 1, args=(Grid_parameter[i]), epsrel=1e-10, limit=100)
print(time.time() - start)

213.64308738708496


$\int_0^1 \tau^2 d\tau\sum \exp \left( -\tau^2 (\widetilde{x}^2 + \widetilde{y}^2)  \right)$


$\widetilde{x} = x/ \alpha$

$\widetilde{y} = y/ \beta$

In [64]:
import numpy as np
from scipy.integrate import quad


def fun(tau, x, y, alpha, beta):
    """
    x: int
    y: int
    alpha: array
    beta: array
    """
    x_til = x/alpha
    y_til = y/beta
    aux = (x_til**2 + y_til**2)
    exp = np.exp(- tau**2*aux)
    
    return (tau**2)*exp.sum()



grid = np.random.random((10, 2))

alpha = np.array([1,2,3])
beta = np.array([4,5,6])

res = np.empty_like(grid)

for i in range(len(grid)):
    res[i] = quad(fun, 0, 1, args=(grid[i][0],grid[i][1], alpha, beta))

In [62]:
grid = np.random.random((10, 2))

alpha = np.array([1,2,3])
beta = np.array([4,5,6])

res = np.empty_like(grid)

for i in range(len(grid)):
    res[i] = quad(fun, 0, 1, args=(grid[i][0],grid[i][1], alpha, beta))

$\int_0^1 \tau^2 d\tau\sum \exp \left( -\tau^2 (\widetilde{x}^2 + \widetilde{y}^2)  \right)$


$\widetilde{x} = x/ \alpha$

$\widetilde{y} = y/ \beta$

$\left( \int_{-\infty}^{+\infty} e^{-x^2} dx \right)$

In [117]:
import numpy as np
import quadpy

np.random.seed(0)
ws = 2 * np.random.random(10) - 1


def f(x):
    out = np.empty((len(ws), *x.shape))
    out0 = np.abs(np.multiply.outer(ws, x))
    out1 = np.multiply.outer(ws, np.exp(x))
    out[ws < 0] = out0[ws < 0]
    out[ws >= 0] = out1[ws >= 0]
    return out


def integrand(x):
    return f(x) * np.log(np.sum(f(x), axis=0))


val, err = quadpy.quad(integrand, -1, +1, epsabs=1.0e-10)
print(val)




[0.3266534  1.44001826 0.68767868 0.30035222 0.18011948 0.97630376
 0.14724906 2.62169217 3.10276876 0.27499376]


In [129]:
out = np.empty((len(ws), *x.shape))

out

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [131]:
out0 = np.abs(np.multiply.outer(ws, x))
out

array([[6.90692641e-310, 4.68479435e-310, 0.00000000e+000, ...,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       ...,
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000]])

In [118]:

    x_til = x[i]/sigma
    y_til = y[i]/sigma


    ax = lambda tau: tau * np.sum(  (M/sigma)*(x_til/(np.sqrt(1 - eta_sq*(tau)**2)))       *  np.exp( - (tau**2/2)*(x_til**2 + y_til**2/(1-eta_sq*(tau)**2)) )     )

    ay = lambda tau: tau * np.sum(  (M/sigma)*(y_til/(np.power(1 - eta_sq*(tau)**2, 3/2))) *  np.exp( - (tau**2/2)*(x_til**2 + y_til**2/(1-eta_sq*(tau)**2)) )     )

    resultado_x[i] = quad_vec(ax, 0, 1)
    resultado_y[i] = quad_vec(ay, 0, 1)


array([ 0.09762701,  0.43037873,  0.20552675,  0.08976637, -0.1526904 ,
        0.29178823, -0.12482558,  0.783546  ,  0.92732552, -0.23311696])

In [190]:
def fx(tau):

    return alphax(tau, Grid_parameter[i][0], Grid_parameter[i][1], Grid_parameter[i][2],Grid_parameter[i][3],Grid_parameter[i][4])


def fy(tau):

    return alphay(tau, Grid_parameter[i][0], Grid_parameter[i][1], Grid_parameter[i][2],Grid_parameter[i][3],Grid_parameter[i][4])



In [195]:
rx = np.zeros([len(grid),2])
ry = np.zeros([len(grid),2])

start = time.time()
for i in range(len(grid)):
    rx[i] = quadpy.quad(fx, 0,1 )
    ry[i] = quadpy.quad(fy, 0,1 )
    
print(time.time() - start)

363.66140723228455


In [196]:
rx

array([[-9.62147667e+14,  3.18154875e+03],
       [-8.50121111e+14,  2.65566675e+03],
       [-7.37659697e+14,  2.19098442e+03],
       ...,
       [ 7.37659697e+14,  2.19098442e+03],
       [ 8.50121111e+14,  2.65566675e+03],
       [ 9.62147667e+14,  3.18154875e+03]])

In [197]:
result_x

array([[-9.62147667e+14,  1.19841693e+03,  0.00000000e+00],
       [-8.50121111e+14,  1.10273129e+03,  0.00000000e+00],
       [-7.37659697e+14,  9.89420267e+02,  0.00000000e+00],
       ...,
       [ 7.37659697e+14,  9.89420267e+02,  0.00000000e+00],
       [ 8.50121111e+14,  1.10273129e+03,  0.00000000e+00],
       [ 9.62147667e+14,  1.19841693e+03,  0.00000000e+00]])

In [198]:
ry

array([[ 9.79941274e+15,  2.78885177e+04],
       [ 9.81327533e+15,  2.63830715e+04],
       [ 9.82544450e+15,  2.51157497e+04],
       ...,
       [-9.82544450e+15,  2.51157497e+04],
       [-9.81327533e+15,  2.63830715e+04],
       [-9.79941274e+15,  2.78885177e+04]])

In [199]:
result_y

array([[ 9.79941274e+15,  1.04244259e+04,  0.00000000e+00],
       [ 9.81327533e+15,  1.08750712e+04,  0.00000000e+00],
       [ 9.82544450e+15,  1.12618053e+04,  0.00000000e+00],
       ...,
       [-9.82544450e+15,  1.12618053e+04,  0.00000000e+00],
       [-9.81327533e+15,  1.08750712e+04,  0.00000000e+00],
       [-9.79941274e+15,  1.04244259e+04,  0.00000000e+00]])